#Intent Classification using LSTM
This use-case provides a demo of how LSTM can be used for Intent classification in texts.

##Workflow:

1.   Understanding the problem
2.   Reading the data and understanding it
3.   Data Preprocessing
4.   Build LSTM model
5.   Train & Evaluate the model

##1. Understanding the problem
Intent Classification is the automated association of text to a specific intention. For example: Let's say you are writing an email to one of the Airlines and the text of the same is 'Can you please cancel my ticket with PNR 123456'. The intent of the customer here is 'Cancellation of Air Ticket'.

The idea of this use case to introduce the concept of Intent classification and how can LSTM be used to solve this.

###Import the necessary libraries
Please load the following packages before you proceed further.


In [ ]:
import numpy as np
import pandas as pd
import nltk

from sklearn.preprocessing import OneHotEncoder as oneHot
from nltk.corpus import stopwords
from nltk import word_tokenize
from string import punctuation
from nltk.stem import PorterStemmer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import BatchNormalization, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy as cce
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.initializers import he_uniform, glorot_uniform
from tensorflow.keras.metrics import AUC
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2
from sklearn.metrics import classification_report

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#Read the train and test datasets with column names as target and text
train= pd.read_csv('/content/bank_train - bank_train.csv',
                       names= ["target", "text"])

test= pd.read_csv('/content/bank_test.csv',
                       names= ["target", "text"])

In [ ]:
train.shape

(79, 2)

In [ ]:
test.shape

(21, 2)

####This is how the data looks like

In [ ]:
train.head(10) #Get Top 10 rows from train dataset

,target,text
0,0,I APPLIED FOR A CREDIT CARD LAST MONTH BUT I D...
1,1,I OPENED A NEW ACCOUNT IN YOUR BANK BUT WHEN I...
2,0,I APPLIED FOR DEBIT OR CREDIT CARD WHEN I OPEN...
3,0,TILL NOW I DID NOT GET ANY DEBIT OR CREDIT CAR...
4,1,I GAVE AN APPLICATION FOR LINKING UNIQUE NUMBE...
5,1,I WANT TO CHANGE THE MOBILE NUMBER THAT WAS PR...
6,0,I LOST MY DEBIT OR CREDIT CARD YESTERDAY . SO ...
7,0,PLEASE BLOCK MY DEBIT OR CREDIT CARD BECAUSE I...
8,1,I GAVE AN APPLICATION FOR LINKING MY MOBILE NU...
9,1,I GAVE AN APPLICATION FOR LINKING MY MOBILE NU...


 Check the number of intents.

In [ ]:
train['target'].value_counts() #Get counts of different types of target variable in train data. We will not be using this anywhere but it is just for the overview of the data.

target
2    29
1    26
0    24
Name: count, dtype: int64

### 3. Preprocessing the Data
We will be doing the following preprocessing steps to get the desired format of the data.

1. Perform One Hot Encoding on the target variable of both train & test datasets.
2. Convert the text into lower case.
3. Tokenize the words.
4. Remove stop words.
5. Perform stemming & normalization.
6. Convert texts into sequences.
7. Pad the sequences.


In [ ]:
encode_target= oneHot().fit(np.array(train.target).reshape(-1,1)) #We perform one hot encoding on the target variable to convert into a matrix of 0s and 1s.

Perform One Hot Encoding on the target variable. The output of this step would be an array with 0s and 1s.

In [ ]:
train_target_encoded= encode_target.transform(np.array(train.target).reshape(-1,1)).toarray()
test_target_encoded= encode_target.transform(np.array(test.target).reshape(-1,1)).toarray()

In [ ]:
train_target_encoded

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0

In [ ]:
test_target_encoded

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

Convert text to lower case

In [ ]:
train["text"]= train.text.map(lambda l: l.lower())
test["text"]= test.text.map(lambda l: l.lower())

In [ ]:
train.head()

,target,text
0,0,i applied for a credit card last month but i d...
1,1,i opened a new account in your bank but when i...
2,0,i applied for debit or credit card when i open...
3,0,till now i did not get any debit or credit car...
4,1,i gave an application for linking unique numbe...


In [ ]:
test.head()

,target,text
0,0,during withdrawing money from account using de...
1,1,the address mentioned on my account is not cor...
2,0,the pin you entered is wrong message was shown...
3,0,i want to change my debit card pin but not abl...
4,0,i am not able to change my debit card security...


Next step is to tokenize the text. We use word_tokenize function from nltk library for this purpose.

In [ ]:
train["text"]= train.text.map(word_tokenize)
test["text"]= test.text.map(word_tokenize)

In [ ]:
#Output of the above exercise looks like this
train["text"]

0     [i, applied, for, a, credit, card, last, month...
1     [i, opened, a, new, account, in, your, bank, b...
2     [i, applied, for, debit, or, credit, card, whe...
3     [till, now, i, did, not, get, any, debit, or, ...
4     [i, gave, an, application, for, linking, uniqu...
                            ...                        
74    [i, want, to, transfer, money, from, my, accou...
75    [i, want, to, transfer, money, from, my, accou...
76    [i, made, a, transaction, with, my, debit, car...
77    [i, made, a, transaction, on, irctc, using, my...
78    [i, applied, for, emi, on, an, online, shoppin...
Name: text, Length: 79, dtype: object

Eliminate stop words. 'english' dictionary from nltk.corpus library is used for this purpose. We also remove punctuation along with the removal of stop words.

In [ ]:
def clean_data_rm_stop(strings, stop_list):
    sw= [str for str in strings if str not in stop_list]
    return sw

stop_words= stopwords.words("english")
rm_punc_stop= list(set(punctuation))+ stop_words #Remove punctuation and stop words

train["text"]= train.text.map(lambda dataframe: clean_data_rm_stop(dataframe, rm_punc_stop))
test["text"]= test.text.map(lambda dataframe: clean_data_rm_stop(dataframe, rm_punc_stop))

Stemming & Normalizing


*   Stemming helps in reducing the word to the root form.
*   Normalizing is the process of transforming text into a standard form. Eg: Gud will be converted to good etc.




In [ ]:
def normalize(text):
    return " ".join(text)

#We use PorterStemmer function from nltk.stem library.
stem_func= PorterStemmer()

train["text"]= train.text.map(lambda s: [stem_func.stem(x) for x in s])
train["text"]= train.text.apply(normalize)

test["text"]= test.text.map(lambda s: [stem_func.stem(x) for x in s])
test["text"]= test.text.apply(normalize)

In [ ]:
train.head()

,target,text
0,0,appli credit card last month get one till alth...
1,1,open new account bank got passbook found middl...
2,0,appli debit credit card open account bank last...
3,0,till get debit credit card appli last month bank
4,1,gave applic link uniqu number bank account las...


Tokenize

In [ ]:
# We use Tokenizer from tensorflow.keras.preprocessing.text library
num_words=10000
text_tokenizer= Tokenizer(num_words)
text_tokenizer.fit_on_texts(train.text) #fit_on_texts - creates the vocabulary index based on word frequency.

tokenized_train_data= text_tokenizer.texts_to_sequences(train.text) #Converting texts to sequences
tokenized_test_data= text_tokenizer.texts_to_sequences(test.text)

In [ ]:
tokenized_train_data

[[8, 15, 2, 12, 18, 24, 39, 9, 68, 131, 132, 52, 8, 15, 2],
 [53, 31, 1, 11, 40, 54, 41, 133, 42, 69, 70],
 [8, 4, 15, 2, 53, 1, 11, 12, 18, 24, 9],
 [9, 24, 4, 15, 2, 8, 12, 18, 11],
 [16, 10, 17, 25, 7, 11, 1, 12, 18, 9, 55],
 [14, 56, 19, 7, 134, 17, 1, 14, 87, 31, 7],
 [88, 4, 15, 2, 71, 14, 89, 4, 15, 2, 72, 29, 90],
 [73, 89, 4, 15, 2, 88, 4, 15, 2, 71, 72, 29, 90],
 [16, 10, 17, 19, 7, 1, 12, 57, 9, 24, 29, 74],
 [16, 10, 17, 19, 7, 1, 12, 18, 24, 135, 72, 1, 75, 7],
 [3, 30, 43, 91, 92, 32],
 [3, 39, 32, 43, 93, 94, 32],
 [95, 33, 1, 26, 96, 20, 21, 3],
 [136, 20, 1, 137, 27, 138, 97, 76],
 [6, 77, 1],
 [139, 77, 6, 1, 44, 21, 3, 5],
 [11, 1, 30, 140, 77, 76, 5],
 [141, 6, 34, 142, 6, 143, 78, 76, 144, 1],
 [145, 74, 3, 20],
 [6, 146, 98, 147, 148, 99, 3, 20],
 [8, 31, 1, 11, 40, 54, 41, 42, 79, 69, 70],
 [35, 22, 5, 4, 15, 2, 45, 58, 59, 2, 100],
 [5, 4, 2, 46, 5, 20, 45, 58, 59, 2, 100],
 [8, 31, 1, 11, 40, 54, 41, 149, 150, 69, 70, 54],
 [21, 151, 5, 3, 44],
 [14,
  8,
  101

Then, we pad the sequences

In [ ]:
#We use pad_sequences from tensorflow.keras.preprocessing.sequence library
train_data= pad_sequences(tokenized_train_data, maxlen= 20, padding= "pre")
test_data= pad_sequences(tokenized_test_data, maxlen= 20, padding= "pre")

In [ ]:
train_data

array([[  0,   0,   0, ...,   8,  15,   2],
       [  0,   0,   0, ...,  42,  69,  70],
       [  0,   0,   0, ...,  18,  24,   9],
       ...,
       [  0,   0,   0, ...,   6,  23,   1],
       [  0,   0,   0, ...,   6, 198,   1],
       [  0,   0,   0, ...,  59,  38,  51]], dtype=int32)

In [ ]:
train_data.shape

(79, 20)

Let's build a 3 dim array. The dimensions are samples, steps and unique words.

In [ ]:
def transform_matrix(data, tokenizer):
    output_shape_mat= [data.shape[0],
                  data.shape[1],
                  tokenizer.word_index.keys().__len__()] #Three dimensional matrix with samples, steps and number of uniques words as each dimension.
    results_data= np.zeros(output_shape_mat) #creates new array with given dimensions.

    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            results_data[i, j, data[i,j]-1]= 1 # In this for loop, we are looping over the shape of the training & test data and assigning the cell of above created zero matrix to 1. We are performing encoding on the unique words to obtain the transformation matrix
    return results_data

trans_matrix_train= transform_matrix(train_data, text_tokenizer) #This will be the matrix on which the lstm model is applied
trans_matrix_test= transform_matrix(test_data, text_tokenizer) #This will be the matrix on which the model is tested

In [ ]:
trans_matrix_train.shape

(79, 20, 199)

In [ ]:
trans_matrix_test.shape

(21, 20, 199)

### 4. Build LSTM Model

Build a class lstm_model_class that has three methods(methods are similar to functions) in a class, they are:



1.   Build a lstm model
2.   Train the created lstm model on the train data
3.   Predict the output on the train data

Note: Building a Class with 3 methods helps in tying all these three functions to the same object at the same instance.

Process of Building a LSTM Model:



1.   Build an embedded layer with dimensions as number of steps and input dimensions.
2.   Build an LSTM layer with number of steps equal to memory units.  
3.   Then, build a dense layer which is fully connected layer that represents a matrix vector multiplication.
4.   Apply the relu function after normalization and scaling of the activations. This is the standard activation function used.
5.   Finally, build the output layer.

Note:
1. activation function for multi-class classification problem - softmax
2. loss function is categorical cross entropy.
3. performace metric - Area under the curve(AUC)
4. optimizer would be the Adam optimizer.





In [ ]:
class lstm_model_class(object):
    def build_lstm_model(self,input_dimensions, op_shape, num_steps, dropout_rate, kernel_reg, bias_reg):
      ip_layer= Input(shape= (num_steps, input_dimensions)) #Define embedded layer with shape as number of steps and input dimensions. Note that both these are input variables to the model.

      lstm_model= LSTM(units= num_steps)(ip_layer) #Make the LSTM layer with number of steps as memory units
      dense_layer_1= Dense(op_shape, kernel_initializer= he_uniform(), #he_uniform draws samples in uniform distribution with -inf to +inf as range.
                   bias_initializer= "zeros",
                   kernel_regularizer= l2(l= kernel_reg),
                   bias_regularizer= l2(l= bias_reg))(lstm_model) # Create the  Dense layer which is the regular deeply connected layer
      int_layer= BatchNormalization()(dense_layer_1) #Normalize and scale activations of the dense layer with BatchNormalization function
      int_layer= relu(int_layer) #This applies the rectified linear unit activation function
      int_layer= Dropout(rate= dropout_rate)(int_layer) #Dropout is used to define Dropout layer that sets input units to 0 with a frequency. Here it is dropout_rate
      output_1= Dense(op_shape, kernel_initializer= glorot_uniform(), #glorot_uniform draws samples in uniform distribution with stddev = sqrt(2 / (fan_in + fan_out)) fan_in is num of units in weight tensor and fan_out is num of output units
             bias_initializer= "zeros",
             kernel_regularizer= l2(l= kernel_reg),
             bias_regularizer= l2(l= bias_reg))(dense_layer_1) # Create another dense layer which is the output of the model.
      output_1= BatchNormalization()(output_1) #Normalize and scale activations of the dense layer with BatchNormalization function
      final_output= softmax(output_1, axis= 1)

      loss_func= cce() # Since it is a multi-class classification problem, categorical crossentropy(cce) is used as the loss function
      perf_metrics= AUC() #our performance metric will be area under the curve
      optimizer= Adam() #we shall use Adam optimizer as our optimizer
      self.final_model= Model(inputs= [ip_layer], outputs= [final_output]) #Build the model with input and output layers
      self.final_model.compile(optimizer= optimizer, loss= loss_func, metrics= [perf_metrics]) #Compiling the keras model

    def train_lstm_model(self,x, y, valid_split, ep):
      self.final_model.fit(x, y, validation_split= valid_split, epochs= ep) #Create the train model


    def predict_lstm_model(self,x):
      return self.final_model.predict(x)    #Create the predict model


Build the model on the necessary inputs.
We define the number of steps as the , output shape and input dimensio appropriately.

In [ ]:
steps= trans_matrix_train.shape[1] #Define the number of steps is usually the number of steps in the train data.
input_dim= trans_matrix_train.shape[2] #Input dimension. Number of unique words in the train data
output_shape= train_target_encoded.shape[1] #Output shape. Usually the same number as the number of classes in the target variable. Here we have 8.
final_model= lstm_model_class()
final_model.build_lstm_model(input_dimensions= input_dim,
                  op_shape= output_shape,
                  num_steps= steps,
                  dropout_rate= 0.5, # Meaning 1 in 2 inputs will be randomly executed.
                  bias_reg= 0.3, # Reduce the bias in the model
                  kernel_reg= 0.3) #Reduce the weights excluding bias.



### 5. Train & Evaluate the model
In our last step, we will train and evaluate the model and check the performance metrics.

In [ ]:
final_model.train_lstm_model(trans_matrix_train, train_target_encoded,
           0.2, 60) #Model takes train data, train target variable, validation split(here it is 80:20) and number of epochs.

Epoch 1/60
2/2 [==============================] - 6s 646ms/step - loss: 4.2994 - auc: 0.3750 - val_loss: 3.8266 - val_auc: 0.2412
Epoch 2/60
2/2 [==============================] - 0s 58ms/step - loss: 4.0997 - auc: 0.4609 - val_loss: 3.7970 - val_auc: 0.2871
Epoch 3/60
2/2 [==============================] - 0s 56ms/step - loss: 3.9239 - auc: 0.5410 - val_loss: 3.7665 - val_auc: 0.3662
Epoch 4/60
2/2 [==============================] - 0s 41ms/step - loss: 3.7191 - auc: 0.6510 - val_loss: 3.7361 - val_auc: 0.4238
Epoch 5/60
2/2 [==============================] - 0s 54ms/step - loss: 3.5520 - auc: 0.7542 - val_loss: 3.7065 - val_auc: 0.4766
Epoch 6/60
2/2 [==============================] - 0s 54ms/step - loss: 3.4217 - auc: 0.8216 - val_loss: 3.6774 - val_auc: 0.5361
Epoch 7/60
2/2 [==============================] - 0s 55ms/step - loss: 3.3043 - auc: 0.8811 - val_loss: 3.6483 - val_auc: 0.5908
Epoch 8/60
2/2 [==============================] - 0s 44ms/step - loss: 3.2144 - auc: 0.9206 - va

In [ ]:
pred_train= encode_target.inverse_transform(final_model.predict_lstm_model(trans_matrix_train)) #Predict on the train matrix and look at the performance
print(classification_report(train.target, pred_train)) #Print the classification report

3/3 [==============================] - 0s 5ms/step
              precision    recall  f1-score   support

           0       0.96      1.00      0.98        24
           1       1.00      0.88      0.94        26
           2       0.94      1.00      0.97        29

    accuracy                           0.96        79
   macro avg       0.97      0.96      0.96        79
weighted avg       0.96      0.96      0.96        79



F1 and weighted avg are excellent. We can now move to implement this model on test data and see how it is performing.

In [ ]:
pred_test= encode_target.inverse_transform(final_model.predict_lstm_model(trans_matrix_test)) #Predict on the test data
print(classification_report(test.target, pred_test)) #Print the classification report

1/1 [==============================] - 0s 32ms/step
              precision    recall  f1-score   support

           0       0.88      0.78      0.82         9
           1       1.00      0.40      0.57         5
           2       0.64      1.00      0.78         7

    accuracy                           0.76        21
   macro avg       0.84      0.73      0.72        21
weighted avg       0.83      0.76      0.75        21



F1 and weighted avg are excellent. We can settle with this model.